In [1]:
import os
import geosoft.gxpy.gdb as gxgdb
import geosoft.gxpy.gx as gx
import pandas as pd
import csv
from collections import Counter
import re
import win32com.client
from tqdm import tqdm
import geosoft.gxapi as gxapi
import datetime
import json # <--- NEW: Import for saving/loading JSON
import shutil
from pathlib import Path

# --- Constants ---
COLOR_RED = '\033[91m'
COLOR_BLUE = '\033[94m'
COLOR_GREEN = '\033[92m'
COLOR_YELLOW = '\033[93m'
RESET = '\033[0m'
UTM_LOOKUP_CACHE_FILE = "utm_zone_lookup_cache.json" # <--- NEW: Define cache file name

In [ ]:
failed_files_path_only = []

# --- Helper Functions (Your existing functions) ---

class HeaderDetectionError(Exception):
    def __init__(self, message, filepath=None):
        super().__init__(message)
        self.filepath = filepath

def is_number(s):
    '''
    detect if s contains a number.
    s (string): a value to check.
    '''
    try:
        float(s)
        return True
    except ValueError:
        return False

def is_wildcard(s):
    '''
    detect if s contains a wildcard.
    s (string): a value to check.
    '''
    return s in ('', '*', None, 'NA', 'null', 'n/a')

def has_same_format(list1, list2):
    '''
    determining if two strings have the same format across all items.

    list1 (list): the first list to compare
    list2 (list): the second list to compare
    '''
    for a, b in zip(list1, list2):
        if is_wildcard(a) or is_wildcard(b):
            continue
        if is_number(a) and is_number(b):
            continue
        if not is_number(a) and not is_number(b):
            continue
        return False
    return True

def return_number(list):
    '''
    finds and returns a number coming after 'UTM' regardless of casing
    list (list): a list containing at least one string
    '''
    match = re.search(r'(?:utm|zone)[^a-zA-Z\d]*?(\d+)', list[0], re.IGNORECASE)
    return match.group(1) if match else None

def looks_like_header(row):
    # Check if majority of the elements are non-numeric strings
    non_numeric_count = 0
    for elem in row:
        if elem is None or (isinstance(elem, str) and elem.strip() == ''):
            non_numeric_count += 1
            continue
        try:
            float(elem)
        except (ValueError, TypeError):
            non_numeric_count += 1
    return non_numeric_count > len(row) / 2

def extract_text_from_doc(doc_path):
    # This function is the primary cause of slowness when repeatedly called.
    # It requires Microsoft Word to be installed.
    word = win32com.client.Dispatch("Word.Application")
    word.Visible = False
    doc = word.Documents.Open(doc_path)
    file_text = doc.Content.Text
    doc.Close()
    word.Quit()
    return file_text

def extract_utm_and_zone(file_text):
    utm = None
    zone = None
    lines = file_text.lower().splitlines()

    for line in lines:
        line_lower = line.lower()

        # First try to extract NAD-83 + Zone like '18N'
        if "utm" in line_lower and "nad" in line_lower:
            utm_match = re.search(r"(nad-?\d+)", line, re.IGNORECASE)
            zone_match = re.search(r"zone\s*(\d+[a-z]?)", line, re.IGNORECASE)

            if utm_match:
                utm = utm_match.group(1).upper()
            if zone_match:
                zone = zone_match.group(1).upper()
            if utm and zone:
                break

        # If not found, fallback to older generic pattern
        if utm is None or zone is None:
            utm_match = re.search(r"utm\s+([a-z0-9]+)", line_lower)
            zone_match = re.search(r"zone\s*\(?\s*(\d+)", line_lower)

            if utm_match and utm is None:
                utm = utm_match.group(1).upper()
            if zone_match and zone is None:
                zone = zone_match.group(1)
            if utm and zone:
                break
            
    return utm, zone

def process_folder(folder_path):
    # Check all .doc files in the folder
    doc_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.doc')]
    
    if not doc_files:
        return None

    for doc_file in doc_files:
        doc_path = os.path.join(folder_path, doc_file)
        try:
            text = extract_text_from_doc(doc_path)
            utm, zone = extract_utm_and_zone(text)
            if utm and zone:
                return (utm, zone)  # Return the first found pair
        except Exception as e:
            print(f"[{os.path.basename(folder_path)}] Error reading {doc_file}: {e}")
    
    return None

def process_all_folders_recursively(root_dir, mode="create"):
    results = {}
    action = "Creating" if mode == "create" else "Updating"
    
    print(f"Scanning for .doc/.txt files in: {root_dir} (This might take a while if not cached)...")
    # Using os.walk for tqdm to show progress for folder processing
    all_folders = [folder_path for folder_path, _, _ in os.walk(root_dir)]
    
    for folder_path in tqdm(all_folders, desc=f"{COLOR_YELLOW}{action} Dictionary of UTM/Zone{RESET}", colour='yellow'):
        res = process_folder(folder_path)
        if res:
            # Store with relative path or full path as key to avoid name conflicts
            # Using full path as key to ensure uniqueness
            results[folder_path] = res
    return results

# --- Cache Management Functions ---
def load_utm_lookup_cache(cache_file_path):
    if os.path.exists(cache_file_path):
        # print(f"{COLOR_BLUE}Loading UTM lookup from cache: {cache_file_path}")
        try:
            with open(cache_file_path, 'r', encoding="utf-8", errors="ignore") as f:
                return json.load(f)
        except json.JSONDecodeError as e:
            print(f"{COLOR_RED}Error decoding cache file '{cache_file_path}': {e}. Recreating cache.")
            os.remove(cache_file_path) # Delete corrupt cache
            return {}
        except Exception as e:
            print(f"{COLOR_RED}Error loading cache '{cache_file_path}', recreating: {e}")
            return {}
    return {}

def save_utm_lookup_cache(utm_data, cache_file_path):
    print(f"{COLOR_BLUE}Saving UTM lookup to cache: {cache_file_path}")
    try:
        with open(cache_file_path, 'w', encoding='utf-8', errors="ignore") as f:
            json.dump(utm_data, f, indent=4)
        print(f"{COLOR_BLUE}Cache saved successfully.")
    except Exception as e:
        print(f"{COLOR_RED}Error saving cache to {cache_file_path}: {e}")
        
def update_utm_lookup_cache(root_directory, cache_file_path):
    """
    Updates (or creates) the UTM cache by re-scanning all folders and adding any new UTM data.
    Avoids duplicate keys by comparing normalized full paths.
    """
    # Load existing cache
    utm_cache = load_utm_lookup_cache(cache_file_path)
    normalized_cache = {os.path.normpath(k).lower(): v for k, v in utm_cache.items()}
    updated = False

    print(f"{COLOR_BLUE}Scanning folders to extract UTM info... (this may take a while)")

    # Re-scan all folders for UTM info
    new_data = process_all_folders_recursively(root_directory, mode="update")

    new_entries = 0
    for folder, utm_info in new_data.items():
        norm_folder = os.path.normpath(folder).lower()
        if norm_folder not in normalized_cache:
            utm_cache[folder] = utm_info
            normalized_cache[norm_folder] = utm_info
            print(f"{COLOR_GREEN}Added new UTM info for folder: {folder}")
            updated = True
            new_entries += 1

    if updated:
        save_utm_lookup_cache(utm_cache, cache_file_path)
        print(f"{COLOR_GREEN}✓ Cache updated with {new_entries} new entr{'y' if new_entries == 1 else 'ies'}.")
    else:
        print(f"{COLOR_YELLOW}No new UTM data found. Cache is already up to date.")

    return utm_cache

# Cleaning up the empty folders in putout directory after all CSV files were created
def delete_empty_folders(output_directory):

    # Variable to track if any folders were deleted
    # This is to ensure we keep checking until no more empty folders are found
    not_done = True

    while not_done:
        # Reset the flag for each iteration
        not_done = False
        # Walk the directory tree bottom-up
        for dirpath, dirnames, filenames in os.walk(output_directory, topdown=False):
            if not dirnames and not filenames:  # no subfolders and no files
                # print(f"Deleting empty folder: {dirpath}")
                os.rmdir(dirpath)
                # Set the flag to True to indicate a folder was deleted and the loop should run again
                not_done = True
    
    print(f"{COLOR_GREEN}Output Directory finished cleaning.")

# --- .doc and .txt parsing functions ---
def parse_txt_header_1(txt_file):
    short_headers = []

    with open(txt_file, 'r', encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    start_index = next((i for i, line in enumerate(lines) if "LENGTH" in line and "MULTIPLIER" in line), None)
    if start_index is None:
        raise ValueError("Header section not found")

    for line in lines[start_index+2:]:
        line = line.strip()
        if not re.match(r"^\w+", line):
            break

        match = re.match(r"(\w+)\s*:?\s+(\d+)\s+(\S+)\s+(.*)", line)
        if not match:
            continue

        field, length, multiplier, comment = match.groups()

        if field.upper() == "BLANK":
            continue

        comment = comment.lower()
        header = ""

        if field.upper() == "EM":
            freq_match = re.search(r'(\d+)', comment)
            freq = freq_match.group(1) if freq_match else ""

            if "coaxial" in comment:
                header = f"Em{freq}coax"
            elif "coplanar" in comment:
                header = f"Em{freq}copl"
            else:
                header = f"Em{freq}"

            if "inphase" in comment:
                header += "p"
            elif "quadrature" in comment:
                header += "q"

        elif field.upper() == "RESISTIVITY":
            freq_match = re.search(r'(\d+)', comment)
            freq = freq_match.group(1) if freq_match else ""
            header = f"Res{freq}"

        elif field.upper() == "POSITIONING":
            freq_match = re.search(r'([a-z0-9]+)', comment, re.IGNORECASE)
            freq = freq_match.group(1) if freq_match else ""

            comment_lower = comment.lower()
            if "easting" in comment_lower:
                header = "X"
            elif "northing" in comment_lower:
                header = "Y"

        else:
            known_map = {
                "LINE": "Line",
                "FLIGHT": "Flt",
                "DATE": "Date",
                "TIME": "Time",
                "MANUAL": "Fid",
                "ALTIMETER": "Alti",
                "MAGNETICS": "Magc",
                "WPM": "Pm"
            }
            header = known_map.get(field.upper(), field.capitalize())

        short_headers.append(header)
    return short_headers

def parse_txt_header_2(file_path):
    keyword_map = {
        'x': 'X', 'y': 'Y', 'fiducial': 'Fid', 'flight number': 'Vol',
        'date': 'Date', 'time': 'Time', '60 hz detect.': 'Hz60',
        'barometric altimeter': 'Altibaro', 'radar altimeter': 'Altiradar',
        'mag corrected': 'Magc', 'mag original': 'Mago',
        'vlflt ( corrected )': 'VLFLT corrected', 'vlflq ( corrected )': 'VLFLQ corrected',
        'vlfot ( corrected )': 'VLFOT corrected', 'vlfoq ( corrected )': 'VLFOQ corrected',
        'vlflt ( original )': 'VLFLT original', 'vlflq ( original )': 'VLFLQ original',
        'vlfot ( original )': 'VLFOT original', 'vlfoq ( original )': 'VLFOQ original',
        'zgps': 'ZGPS', 'k corrected (potassium)': 'K corrected',
        'th corrected (thorium)': 'Th corrected', 'u corrected (uranium)': 'U corrected',
        'tc corrected (total counts)': 'Total Count corrected',
        'tcexp corrected (total count exposure)': 'Total count Exposure corrected',
        'k original (potassium)': 'K original', 'th original (thorium)': 'Th original',
        'u original (uranium)': 'U original', 'tc original (total counts)': 'Total Count original',
        'longitude': 'Longitude', 'latitude': 'Latitude',
        'mag igrf': 'MAG igrf', 'zpnav': 'Zpnav', 'mag' : 'MAG',
        'vlf line quadrature': 'VLFLQ', 'vlf line total': 'VLFLT',
        'vlf ortho quadrature': 'VLFOQ', 'vlf ortho total': 'VLFOT',
    }

    em_pattern = re.compile(
        r'F\d+\s*-\s*(\d+)\s*-\s*(corrected|original)\s+(coaxial|coplanar)\s+(inphase|quadrature)',
        re.IGNORECASE
    )
    resistivity_pattern = re.compile(r'resistivity\s*\(\s*[-]?(\d+)\s*Hz-?\s*\)', re.IGNORECASE)

    headers = []

    with open(file_path, 'r', encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not re.match(r'^\d+\s', line):
                continue

            parts = line.split(maxsplit=2)
            if len(parts) == 2:
                field_desc = parts[1].strip().lower()
            elif len(parts) == 3:
                field_desc = parts[2].strip().lower()
            else:
                continue

            header = None
            em_match = em_pattern.search(field_desc)
            if em_match:
                freq = em_match.group(1)
                geom = 'coax' if 'coax' in em_match.group(3).lower() else 'copl' # Corrected group index
                phase = 'p' if 'inphase' in em_match.group(4).lower() else 'q' # Corrected group index
                header = f"Em{freq}{geom}{phase}"

            elif "resistivity" in field_desc:
                res_match = resistivity_pattern.search(field_desc)
                if res_match:
                    res_freq = res_match.group(1)
                    header = f"Res{res_freq}"

            else:
                for key, val in keyword_map.items():
                    if field_desc == key:
                        header = val
                        break
                    if field_desc.startswith(key + ' '):
                        header = val
                        break

            if not header:
                header = "Unknown"

            headers.append(header)
    return headers

def parse_doc_header_1(file_path):
    headers = []
    in_format_section = False
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            if "SURVEY DATA FORMAT" in line.upper():
                in_format_section = True
                continue
            if in_format_section and (line.startswith("PARAMETER DESCRIPTIONS") or line.startswith("*****")):
                break

            if in_format_section:
                match = re.match(r'^\s*\d+\s+(\S+)', line)
                if match:
                    param = match.group(1)
                    headers.append(param)
    return headers

def header_detection(xyz_path, missing_counter, utm_lookup_dict): # <--- NEW: Pass utm_lookup_dict
    xyz_directory = os.path.dirname(xyz_path)
    txt_doc_file_path = None
    
    # Check utm_lookup_dict first (efficient for .doc files)
    # This is a bit tricky since header_detection might be called for individual XYZ files
    # whose parent folder might already be in utm_lookup_dict.
    # For XYZ files, header info is often in a .txt or .doc *within the same folder*,
    # not necessarily just the parent folder name.
    
    # Find matching .txt or .doc file in the same directory as the XYZ file
    for filename in os.listdir(xyz_directory):
        name, ext = os.path.splitext(filename)
        # Check if the text/doc file's name contains the XYZ file's name
        # or if it's the folder's name (common for summary docs)
        if (os.path.splitext(os.path.basename(xyz_path))[0].lower() in name.lower() or
            os.path.basename(xyz_directory).lower() in name.lower()) and ext.lower() in ['.txt', '.doc']:
            txt_doc_file_path = os.path.join(xyz_directory, filename)
            break
            
    if txt_doc_file_path is None:
        missing_counter[0] += 1
        failed_files_path_only.append(xyz_path)
        raise HeaderDetectionError(f"No matching TXT/DOC file found for: {xyz_path}", xyz_path)

    print(f"Attempting to parse header from: {txt_doc_file_path}")

    ext = os.path.splitext(txt_doc_file_path)[1].lower()

    if ext == '.txt':
        try:
            headers = parse_txt_header_1(txt_doc_file_path)
            if headers: return headers
        except Exception as e:
            pass # print(f"Error in parse_txt_header_1: {e}")

        try:
            headers = parse_txt_header_2(txt_doc_file_path)
            if headers: return headers
        except Exception as e:
            pass # print(f"Error in parse_txt_header_2: {e}")

    elif ext == '.doc':
        # Here we can directly use the cached data for the folder if available
        # But this function is about header parsing, not UTM/Zone lookup for entire folder.
        # So we'll stick to extracting from the doc here, but only if it's the *specific* doc
        # that serves as a header file for the XYZ.
        try:
            headers = parse_doc_header_1(txt_doc_file_path)
            if headers: return headers
        except Exception as e:
            pass # print(f"Error in parse_doc_header_1: {e}")

    print(f"{COLOR_RED}No headers found by either parser for {os.path.basename(xyz_path)} in its associated text/doc file.")
    return None

def convert_gdb(gdb_file_path, csv_directory):
    """
    This function converts a Geosoft database (.gdb) to a CSV file.
    :param gdb_file_path: The path to the GDB file.
    :param csv_directory: The directory to save the CSV file.
    """
    try:
        os.makedirs(csv_directory, exist_ok=True)

        with gxgdb.Geosoft_gdb.open(gdb_file_path) as gdb:
            lines = gdb.list_lines()
            channels = gdb.list_channels()
            gdb_data = []

            has_time_utc = "time_utc" in channels

            for line in lines:
                channel_data = {}
                max_rows = 0

                for channel in channels:
                    try:
                        values, _ = gdb.read_channel(line, channel)
                        values = list(values)
                        channel_data[channel] = values
                        max_rows = max(max_rows, len(values))
                    except Exception as e:
                        print(f"{COLOR_RED}Skipping column: {channel} in line: {line} because: {e}")
                        channel_data[channel] = []

                for channel in channels:
                    if len(channel_data[channel]) < max_rows:
                        channel_data[channel] += [None] * (max_rows - len(channel_data[channel]))

                for i in range(max_rows):
                    row = {}
                    if has_time_utc:
                        row["L1010"] = channel_data["time_utc"][i]
                    for channel in channels:
                        if channel != "time_utc":
                            row[channel] = channel_data[channel][i]
                    gdb_data.append(row)
                    
            # Load UTM lookup cache and normalize keys
            with open(UTM_LOOKUP_CACHE_FILE, 'r') as f:
                raw_cache = json.load(f)
                utm_lookup_dict = {os.path.normpath(k).lower(): v for k, v in raw_cache.items()}

            # Normalize the folder path of the input GDB file
            xyz_folder_path = os.path.normpath(os.path.dirname(gdb_file_path)).lower()

            # Try to find UTM and Zone info for this folder path
            utm_zone = None
            if xyz_folder_path in utm_lookup_dict:
                utm_zone = utm_lookup_dict[xyz_folder_path]
            else:
                # Fallback: check if any cached folder path is a prefix of this folder path
                for cached_path, value in utm_lookup_dict.items():
                    if xyz_folder_path.startswith(cached_path):
                        if isinstance(value, list) and len(value) == 2:
                            utm_zone = value
                            break

            # If not found, fill with string 'NaN'
            if utm_zone is None:
                utm_zone = ['NaN', 'NaN']

            # Prepare the header row
            header = []

            # If gdb_data is a list of dicts, convert first dict keys to header (adjust if needed)
            if isinstance(gdb_data[0], dict):
                header = list(gdb_data[0].keys())
            else:
                # If gdb_data is a list of lists, you may already have header elsewhere or need to define it
                # For this example, just create an empty header to be extended below
                header = []

            # Ensure 'L1010' is added if 'time_utc' exists
            if has_time_utc and 'L1010' not in header:
                header.append("L1010")

            # Append all channels except 'time_utc' (avoid duplicates)
            for ch in channels:
                if ch != "time_utc" and ch not in header:
                    header.append(ch)

            # Append 'UTM' and 'Zone' to header if not already there
            if 'UTM' not in header:
                header.append('UTM')
            if 'Zone' not in header:
                header.append('Zone')

            # Append UTM and Zone values to each row in gdb_data
            updated_data = []
            for row in gdb_data:
                if isinstance(row, dict):
                    # Add or update UTM and Zone keys
                    row['UTM'] = utm_zone[0]
                    row['Zone'] = utm_zone[1]
                    updated_data.append(row)
                else:
                    # Assuming row is a list, append utm_zone values
                    updated_data.append(row + utm_zone)

            gdb_data = updated_data

            dataframe = pd.DataFrame(gdb_data)

            if has_time_utc:
                expected_order = ["L1010"] + [col for col in dataframe.columns if col != "L1010"]
                dataframe = dataframe[expected_order]

            csv_file = os.path.join(csv_directory, f"{os.path.splitext(os.path.basename(gdb_file_path))[0]}_gdb.csv")
            dataframe.to_csv(csv_file, index=False, sep=",", encoding="utf-8", escapechar=None, quoting=csv.QUOTE_MINIMAL)

            print(f"{COLOR_BLUE}Successfully converted {gdb_file_path} to {csv_file}")

    except Exception as e:
        print(f"{COLOR_RED}ERROR in processing {gdb_file_path} because: {e}")
        return e

def convert_xyz_to_csv(xyz_file_path, csv_directory, missing_counter, mismatching_files_counter, utm_lookup_dict): # <--- NEW: Pass utm_lookup_dict
    """
    Converts a Geosoft XYZ file to a CSV file without extra quotes around values.
    :param xyz_file_path: The path to the XYZ file.
    :param csv_directory: The directory to save the CSV file.
    """
    os.makedirs(csv_directory, exist_ok=True)
    csv_file_name = os.path.splitext(os.path.basename(xyz_file_path))[0] + ".csv"
    csv_file_path = os.path.join(csv_directory, csv_file_name)
    print(csv_file_path)

    with open(xyz_file_path, encoding="utf-8", errors="ignore") as xyz_file:
        lines = xyz_file.readlines()

    print(f"Loaded: {xyz_file_path}")

    lengths = [len(s.strip().split()) for s in lines]
    counter = Counter(lengths)
    most_common_length = counter.most_common(1)[0][0]
    normalized_lines = [s.strip().split() for s in lines if s.strip()]

    header = None # Initialize header
    header_line = "" # Initialize header_line
    header_source_line_clean = None # Initialize for the try/except block

    try:
        header_search = [s for s in lines if len(s.strip().split()) > most_common_length and '---' not in s and '===' not in s]
        if header_search:
            header_source_line = header_search[-1]
            if ',' in header_source_line:
                if header_source_line.startswith('/'):
                    header_source_line = header_source_line[1:] # Removes the first /
                header_source_line_clean = header_source_line.strip() # Removes white spaces
                header_line = [item.strip() for item in header_source_line_clean.split(',')]
            else:
                header_line = header_search[-1].strip().split()
                header_source_line = header_search[-1]
                header_source_line_clean = header_source_line.strip()
    except Exception: # Catch any error during header_search
        pass # Will proceed to header_detection if header_line is still None

    data_setup = [s for s in lines if len(s.strip().split()) == most_common_length and '---' not in s and '===' not in s]

    if not data_setup:
        print(f"{COLOR_RED}No data lines found with consistent length in {xyz_file_path}.")
        return

    last_row = data_setup[-1].strip().split()
    first_row = data_setup[0].strip().split()

    if header is None: # If header not determined from initial search
        if header_line: # If we found a candidate header_line from the initial search
            header_vs_data = len(header_line) - len(last_row)
            if header_vs_data >= 0: # Ensure index is not negative
                header = header_line[header_vs_data:]
            else: # If header_line is shorter than data, it's likely not the right header
                header = header_detection(xyz_file_path, missing_counter, utm_lookup_dict) # Fallback to external detection
        else: # No header found from initial search, go straight to external detection
            header = header_detection(xyz_file_path, missing_counter, utm_lookup_dict)

    data = []

    # Re-evaluate data processing logic (simplified based on typical XYZ structures)
    # The original logic for has_same_format(last_row, first_row) and finding the start of data
    # seems overly complex and potentially error-prone for typical XYZ files.
    # Usually, if there's a header, data starts immediately after it.
    # The most common length lines are usually the data.

    # Simpler approach:
    # If a valid header was found, data starts right after the header line.
    # If no header was found internally, assume all most_common_length lines are data.

    if looks_like_header(header):
        header = header
    else:
        header = None
    
    # Single format case: all data lines same format
    data = [s.strip().split() for s in data_setup if s.strip()]
    # Only assign header if not already set or if header length mismatches data length
    if (not header) or (len(header) != len(data[0])):
        if header_line and len(header_line) == len(data[0]):
            header = header_line
        else:
            try:
                # Try to detect header externally
                header_candidate = header_detection(xyz_file_path, missing_counter, utm_lookup_dict)
            except HeaderDetectionError as e:
                # Re-raise so it exits the function
                raise e

            # If header was found but doesn't match data, raise error 
            if header_candidate:
                if len(header_candidate) != len(data[0]):
                    mismatching_files_counter [0] += 1
                    raise HeaderDetectionError(
                        f"Header found from external source (.txt or .doc file), but its length ({len(header_candidate)}) "
                        f"does not match data length ({len(data[0])}) in file: {xyz_file_path}",
                        xyz_file_path
                    )
                else:
                    header = header_candidate
            
    # Find lines that weren't used in header or data
    used_lines_set = set(tuple(row) for row in (data + ([header] if header else [])))
    unused_lines = [original_line for original_line, split_line in zip(lines, normalized_lines)
                    if tuple(split_line) not in used_lines_set]

    # Ensure header is a list and not a tuple (handle (None, None) from header_detection)
    if isinstance(header, tuple):
        # If header is (None, None), treat as missing header
        if all(h is None for h in header):
            if data and len(data) > 0:
                header = [f"Col{i+1}" for i in range(len(data[0]))]
            else:
                header = ["Col1", "Col2", "Col3"]
        else:
            header = list(header)

    # Load the cache file and normalize paths
    with open(UTM_LOOKUP_CACHE_FILE, 'r') as f:
        raw_cache = json.load(f)
        utm_lookup_dict = {os.path.normpath(k).lower(): v for k, v in raw_cache.items()}

    # Get the folder path of the XYZ file
    xyz_folder_path = os.path.normpath(os.path.dirname(xyz_file_path)).lower()

    # Try to find UTM and Zone from dictionary using file path substring
    utm_zone = None

    # Prioritize lookup by exact folder path
    if xyz_folder_path in utm_lookup_dict:
        utm_zone = utm_lookup_dict[xyz_folder_path]
    else:
        # Fallback to looking by folder name
        for folder_path_in_cache, value in utm_lookup_dict.items():
            if xyz_folder_path.startswith(folder_path_in_cache):
                if isinstance(value, list) and len(value) == 2:
                    utm_zone = value
                    break

    # Add headers for UTM and Zone
    if header and 'UTM' not in header and 'Zone' not in header:
        header.extend(['UTM', 'Zone'])
    elif not header:
        header = ['Col1', 'Col2', 'Col3', 'UTM', 'Zone']

    # Append to each row of data
    if utm_zone and isinstance(utm_zone, list) and len(utm_zone) == 2:
        utm_val, zone_val = utm_zone
        data = [row + [utm_val, zone_val] for row in data]
    else:
        data = [row + ['', ''] for row in data]


    # --- Handle missing values: replace empty strings or None with 'NaN' ---
    missing_rows = 0
    def replace_missing(row):
        nonlocal missing_rows
        cleaned = []
        row_missing = False
        for val in row:
            if val is None or (isinstance(val, str) and val.strip() == ''):
                cleaned.append('NaN')
                row_missing = True
            else:
                cleaned.append(val)
        if row_missing:
            missing_rows += 1
        return cleaned
    data = [replace_missing(row) for row in data]
    if missing_rows > 0:
        print(f"{COLOR_GREEN}Info: {missing_rows} row(s) in {xyz_file_path} had missing values replaced with 'NaN'.")

    # This removes all of the potentially extra commas that may exist in the header
    joined = ",".join(header)
    header = [col.strip() for col in joined.split(",") if col.strip()]

    print(f"Saving CSV to directory: {csv_file_path}")
    with open(csv_file_path, 'w', newline='', encoding="utf-8", errors="ignore") as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quoting=csv.QUOTE_NONE, escapechar="\\");

        writer.writerow(header)
        writer.writerows(data)

    print(f"{COLOR_BLUE}Conversion Complete, file placed in: {csv_file_path}")
    clean_up_csv(csv_file_path)


def clean_up_csv(csv_path): # Changed parameter name for clarity
    """
    Cleans up the CSV file by removing escape characters (slashes) that are added during the conversion process.
    :param csv_path: The path to the CSV file.
    """
    try:
        with open(csv_path, 'r', newline='', encoding="utf-8", errors="ignore") as csv_file: # Read with newline='' to avoid issues
            lines = csv_file.readlines()

        cleaned_lines = [line.replace("\\", "") for line in lines]

        with open(csv_path, 'w', newline='', encoding="utf-8", errors="ignore") as csv_file: # Write with newline=''
            csv_file.writelines(cleaned_lines)

        print(f"{COLOR_BLUE}CSV File Cleaned up: {csv_path}")
    except Exception as e:
        print(f"{COLOR_RED}Error cleaning up CSV {csv_path}: {e}")


def csv_carry_over(csv_file_path, output_directory):
    """
    Carries over an existing CSV file to the output directory with UTM and Zone addition modification.
    :param csv_file_path: The path to the input CSV file.
    :param output_directory: The directory to save the converted CSV file.
    """
    os.makedirs(output_directory, exist_ok=True)
    csv_file_name = os.path.basename(csv_file_path)
    output_csv_path = os.path.join(output_directory, csv_file_name)

    try:
        df = pd.read_csv(csv_file_path, encoding="utf-8")
        header = df.columns.tolist()  # Convert Index to list
        data = df.values.tolist()
        
        # Load the cache file and normalize paths
        with open(UTM_LOOKUP_CACHE_FILE, 'r') as f:
            raw_cache = json.load(f)
            utm_lookup_dict = {os.path.normpath(k).lower(): v for k, v in raw_cache.items()}

        # Get the folder path of the CSV file
        csv_folder_path = os.path.normpath(os.path.dirname(csv_file_path)).lower()

        # Try to find UTM and Zone from dictionary using file path substring
        utm_zone = None

        # Prioritize lookup by exact folder path
        if csv_folder_path in utm_lookup_dict:
            utm_zone = utm_lookup_dict[csv_folder_path]
        else:
            # Fallback to looking by folder name
            for folder_path_in_cache, value in utm_lookup_dict.items():
                if csv_folder_path.startswith(folder_path_in_cache):
                    if isinstance(value, list) and len(value) == 2:
                        utm_zone = value
                        break

        # Add headers for UTM and Zone
        if header and 'UTM' not in header and 'Zone' not in header:
            header.extend(['UTM', 'Zone'])
        elif not header:
            header = ['Col1', 'Col2', 'Col3', 'UTM', 'Zone']

        # Append to each row of data
        if utm_zone and isinstance(utm_zone, list) and len(utm_zone) == 2:
            utm_val, zone_val = utm_zone
            data = [row + [utm_val, zone_val] for row in data]
        else:
            data = [row + ['', ''] for row in data]

        print(f"Saving CSV to directory: {output_csv_path}")
        with open(output_csv_path, 'w', newline='', encoding="utf-8", errors="ignore") as csv_file:
            writer = csv.writer(csv_file, delimiter=',', quoting=csv.QUOTE_NONE, escapechar="\\");

            writer.writerow(header)
            writer.writerows(data)

        print(f"{COLOR_BLUE}Carry Over Complete, file placed in: {csv_file_path}")
    except Exception as e:
        print(f"{COLOR_RED}Error in carrying over CSV {csv_file_path}: {e}")


def file_finder(root_directory, output_directory, utm_lookup_dict): # <--- NEW: Pass utm_lookup_dict
    """
    This function finds all GDB and XYZ files in the given directory and converts them to CSV files.
    :param root_directory: The root directory to search for GDB/XYZ files.
    :param output_directory: The base directory to save the CSV files.
    :param utm_lookup_dict: Dictionary containing pre-extracted UTM/Zone info.
    """
    gdb_found = 0
    xyz_found = 0
    csv_found = 0
    gdb_converted = 0
    gdb_convertion_errors = 0
    xyz_converted = 0
    csv_carried = 0
    csv_carry_error = 0
    missing_files_counter = [0]
    mismatching_files_counter = [0]
    unknown_error_counter = 0
    failed_files = []

    # Initialize Geosoft context (this only needs to happen once)
    # Setup Geosoft context if it doesn't already exist
    try:
        gx_context = gx.GXpy()  # try to create context
    except gxapi.GXAPIError as e:
        if "already been created" in str(e):
            pass

    print(f"\n{COLOR_BLUE}--- Starting file conversion ---")
    # Recursively loops through all folders and subfolders finding all GDB/XYZ files
    # Using tqdm for progress on file iteration
    all_folders = [folder for folder, _, _ in os.walk(root_directory)]

    for folder in tqdm(all_folders, desc=f"{COLOR_YELLOW}Processing .gdb and .xyz files{RESET}", colour='yellow'):
        # We need to iterate over files within the current folder for tqdm to count items correctly per folder
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            # Create mirrored output directory path for the current file's folder
            rel_path = os.path.relpath(folder, root_directory)
            output_dir = os.path.join(output_directory, rel_path)
            os.makedirs(output_dir, exist_ok=True) # Ensure output dir exists for current file

            if file.lower().endswith(".gdb"):
                gdb_found += 1
                try:
                    convert_gdb(file_path, output_dir)
                    gdb_converted += 1
                except Exception as ex:
                    gdb_convertion_errors += 1
                    print(f'{COLOR_RED}❌ Fail to extract {file_path} because: {ex}')
                    failed_files.append(f'❌ Failed to extract: {file_path} - Reason: {ex}')
                    failed_files_path_only.append(file_path)

            elif file.lower().endswith(".xyz"):
                xyz_found += 1
                try:
                    convert_xyz_to_csv(file_path, output_dir, missing_files_counter, mismatching_files_counter, utm_lookup_dict) # <--- Pass utm_lookup_dict
                    xyz_converted += 1
                except HeaderDetectionError as hde:
                    error_msg = (f"❌ Header Creation Error: {hde}")
                    print(error_msg)
                    failed_files.append(error_msg)
                except Exception as ex:
                    unknown_error_counter += 1
                    print(f'{COLOR_RED}❌ Fail to extract {file_path}') 
                    failed_files.append(f'{COLOR_RED}❌ Failed to extract: {file_path} - Reason: {ex}')
                    failed_files_path_only.append(file_path)

            elif file.lower().endswith(('.csv')):
                csv_found += 1
                try:
                    csv_carry_over(file_path, output_dir)
                    csv_carried += 1
                except Exception as ex:
                    csv_carry_error += 1
                    print(f'{COLOR_RED}❌ Fail to carry over {file_path} because: {ex}')
                    failed_files.append(f'{COLOR_RED}❌ Failed to carry over: {file_path} - Reason: {ex}')
                    failed_files_path_only.append(file_path)

    # Save failed file paths to a .txt file with error code
    os.makedirs(output_directory, exist_ok=True)
    with open(os.path.join(output_directory, "failed_files_path.txt"), "a", encoding="utf-8", errors="ignore") as f:
        f.write("\n--- New run: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ---\n")
        for path in failed_files:
            f.write(path + "\n")

    # Save failed file paths to a .txt file with only paths
    os.makedirs(output_directory, exist_ok=True)
    with open(os.path.join(output_directory, "failed_files_pathsOnly.txt"), "a", encoding="utf-8", errors="ignore") as f:
        for path in failed_files_path_only:
            f.write(path + "\n")

    # Print summary
    print(f"\n{COLOR_BLUE}=== Conversion Summary ===")
    print(f"GDB Files:")
    print(f" - Files Found: {gdb_found}")
    print(f" - Files Converted: {gdb_converted}")
    print(f" - Files Failed to Convert: {gdb_convertion_errors}")
    print(f"-" * 20)
    print(f"XYZ Files:")
    print(f" - Files Found: {xyz_found}")
    print(f" - Files Converted: {xyz_converted}")
    print(f" - Missing TXT/DOC files for header detection: {missing_files_counter[0]}")
    print(f" - Header and Data Mismatching files: {mismatching_files_counter[0]}")
    print(f" - Unspecified error: {unknown_error_counter}")
    print(f"-" * 20)
    print(f"CSV Files:")
    print(f" - Files Found: {csv_found}")
    print(f" - Files Carried Over: {csv_carried}")
    print(f" - Files Failed to Carry Over: {csv_carry_error}")
    print(f"-" * 20)
    print(f"{COLOR_BLUE}Conversion process completed.")


### Problematic File Gatherer Code
# This code will take a file that contains problematic files, and copy them over to a new folder
# so that the user can easily access them and fix them.
# It also creates a log of how many files were copied over, and their locations.

def copy_problematic_folders(problem_file_path, copied_directory):
    # Open the file and read all lines, stripping whitespace, and convert to Path objects
    with open(problem_file_path, 'r') as f:
        problematic_files = [Path(line.strip()) for line in f if line.strip()]
        
    # Check that the file has file paths
    if not problematic_files:
        print("No problematic files found.")
        return

    # Infer the common project root automatically
    project_root = Path(os.path.commonpath([str(f) for f in problematic_files]))
    if project_root.is_file():
        project_root = project_root.parent

    # Ensure destination base exists
    copied_directory = Path(copied_directory)
    copied_directory.mkdir(parents=True, exist_ok=True)
    
    # Initialize counters
    counters = {'.xyz': 0, '.gdb': 0, '.txt': 0, '.doc': 0}
    
    for file in problematic_files:
        # Check to see if the file exist in your directory
        if not file.exists():
            print(f"File does not exist: {file}")
            continue
        
        folder_to_copy = file.parent
        
        # Prepare list: the actual problem file (.gdb or .xyz from the .txt file) 
        # and prepares any extra .txt/.doc/.docx files in the same folder
        files_to_copy = [file] + [
            f for f in folder_to_copy.iterdir()
            if f.suffix.lower() in {'.txt', '.doc'} and f.is_file()
        ]
        
        for f in files_to_copy:
            # Compute relative path from project root
            relative_path = f.relative_to(project_root)
            destination_path = copied_directory / relative_path

            # Ensure parent exists, then copy the file
            destination_path.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(f, destination_path)
            
            # Increment counter if extension matches
            ext = f.suffix.lower()
            if ext in counters:
                counters[ext] += 1
            
            # Uncomment this if you want this to tell you every file that was copied over
            # Commented out to reduce the output lines
            # print(f"Copied: {f} -> {destination_path}")

    print("Copying complete.")
    print("Summary of files copied:")
    
    for ext, count in counters.items():
        print(f"  {ext}: {count}")


### Error Checker Code

# This is the error finder code that goes through the produced results folder, going through each file and finding the files that contain NaN values for the UTM and Zone values, 
# missing UTM and Zone labels in headers, or if the second row has NaN values in the UTM and/or Zone, as well as the files that contain unknown header values. 
# It will find these problamtic files and document them in their respective error TXT files so the user can simply go through that and see the problemtic files. 

def check_csvs(folder_path):

    missing_utm_zone = []
    missing_second_row = []
    nan_utm_zone = []
    reading_error = []

    print(f"Scanning folder: {folder_path}\n")
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.csv'):
                file_path = os.path.join(root, file)
                try:
                    # Read just the first two rows
                    df = pd.read_csv(file_path, nrows=2, low_memory=False)

                    # Normalize column names
                    normalized_columns = [col.strip().lower() for col in df.columns]

                    if 'utm' not in normalized_columns or 'zone' not in normalized_columns:
                        missing_utm_zone.append("[Missing 'utm' or 'zone' in header] {file_path}")
                        continue

                    utm_col = df.columns[normalized_columns.index('utm')]
                    zone_col = df.columns(normalized_columns.index('zone'))

                    # Check if second row has NaN in utm or zone
                    if len(df) < 2:
                        missing_second_row.append("[Missing second row] {file_path}")
                    elif str(df.at[1, utm_col]).strip().lower() == "nan" or str(df.at[1, zone_col]).strip().lower() == "nan":
                        nan_utm_zone.append("[NaN in UTM or Zone Data] {file_path}")

                except Exception as e:
                    reading_error.append(f"[Error reading file] {file_path} — {str(e)}")
                    
    print('Scanning complete.')

    error_logs = {"Missing_UTM_Zone": missing_utm_zone,
                  "Missing_Second_Row": missing_second_row,
                  "NaN_UTM_Zone": nan_utm_zone,
                  "Reading_Error": reading_error}

    for error_type, items in error_logs.items():
        os.makedirs(folder_path, exist_ok=True)
        with open(os.path.join(folder_path, f"{error_type}.txt"), "a", encoding="utf-8", errors="ignore") as f:
            f.write("\n--- New run: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ---\n")
            for item in items:
                f.write(item + "\n")

    print(f"\n{COLOR_BLUE}Error logs saved in {folder_path} for further review.")

def check_unknown_in_header(folder_path):

    unknown_columns = []
    reading_error = []

    print(f"Scanning folder: {folder_path}\n")
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.csv'):
                file_path = os.path.join(root, file)
                try:
                    # Read only the header row (no data rows)
                    df = pd.read_csv(file_path, nrows=0)

                    # Check each column name for "unknown" (case-insensitive)
                    unknown_columns = [col for col in df.columns if "unknown" in col.lower()]
                    if unknown_columns:
                        unknown_columns.append("[Unknown column(s)] {file_path}")
                        for col in unknown_columns:
                            unknown_columns.append[f" - {col}"]

                except Exception as e:
                    reading_error.append("[Error reading file] {file_path} — {str(e)}")
                    
    print('Scanning complete.')

    error_logs = {"Unknown_Columns": unknown_columns,
                  "Reading_Error": reading_error}
    
    for error_type, items in error_logs.items():
        os.makedirs(folder_path, exist_ok=True)
        with open(os.path.join(folder_path, f"{error_type}.txt"), "a", encoding="utf-8", errors="ignore") as f:
            f.write("\n--- New run: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " ---\n")
            for item in items:
                f.write(item + "\n")

    print(f"\n{COLOR_BLUE}Unknown columns and reading errors saved in {folder_path} for further review.")

In [ ]:
# --- Main Execution Block ---
if __name__ == "__main__":

    # Root directory with the .xyz/.gdb files
    # IMPORTANT: Set this to the parent directory that contains ALL your DP folders    
    root_directory = r'C:\Users\ishpr\Desktop\Mineral Team\Preprocessing Team\Dataset\DP200002\Chunk 1'
    
    # Mimicked directory that will be created to hold the .csv files
    # This directory does not need to exist, it will create everything for you in the desired path 
    output_directory = r'C:\Users\ishpr\Desktop\Mineral Team\Preprocessing Team\Converted_Files'

    # This is the .txt file created by the script where the problematic files pathways ONLY are going to be stored 
    # (Replace with output_directory\problem_files.txt in the future)
    problem_file_path = r'C:\Users\ishpr\Desktop\Mineral Team\Preprocessing Team\Converted_Files\failed_files_pathsOnly.txt'

    # This is the new folder to copy the problematic files into so they can be shared
    # (It doesn't need to exist it wll be created when this is run)
    copied_directory = r'C:\Users\ishpr\Desktop\Mineral Team\Preprocessing Team\Dataset\Copied_Problematic_Files'

    
    
    # --- Implement Caching for UTM Lookup ---
    utm_lookup = load_utm_lookup_cache(UTM_LOOKUP_CACHE_FILE)
 
    if not utm_lookup:
        print(f"{COLOR_RED}Cache not found or empty. Processing all .doc files for UTM/Zone info. This may take a while...")
        # Make sure root_directory for this step is correct for scanning ALL relevant .doc files
        # It needs to be the root of your project structure where folders like DP200601, DP200102 etc. reside.
        utm_lookup = process_all_folders_recursively(root_directory, mode="create")
        save_utm_lookup_cache(utm_lookup, UTM_LOOKUP_CACHE_FILE)
        print(f"{COLOR_BLUE}UTM lookup processing complete and cached.")
    else:
        print(f"{COLOR_BLUE}Cache file loaded: {UTM_LOOKUP_CACHE_FILE}")
        # Updates the UTM cache (adds only new folders)
        utm_lookup = update_utm_lookup_cache(root_directory, UTM_LOOKUP_CACHE_FILE)

    # Now proceed with file conversion using the loaded/generated utm_lookup
    # Ensure the root_directory for file_finder also covers all files you want to convert
    file_finder(root_directory, output_directory, utm_lookup)
    
    # This will process the output directory and delete any folders that have no files or folders inside it
    delete_empty_folders(output_directory)

    # P.S. The failed_files_path.txt file will be empty if no files failed to convert.
    # P.P.S. The output_directory will be created if it doesn't exist, and the converted files will be placed inside it.



    ### Problematic File Gatherer Code Runner
    copy_problematic_folders(problem_file_path, copied_directory)



    ### Running the error checker to find any problematic files in the output directory
    check_csvs(output_directory)
    check_unknown_in_header(output_directory)